<h2 id="30-形態素解析結果の読み込み">30. 形態素解析結果の読み込み</h2>
<p>形態素解析結果（neko.txt.mecab）を読み込むプログラムを実装せよ．ただし，各形態素は表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をキーとするマッピング型に格納し，1文を形態素（マッピング型）のリストとして表現せよ．第4章の残りの問題では，ここで作ったプログラムを活用せよ．</p>


In [3]:
# テキストのダウンロード
!wget -nc https://nlp100.github.io/data/neko.txt -P data/

--2023-04-18 22:13:50--  https://nlp100.github.io/data/neko.txt
Resolving nlp100.github.io (nlp100.github.io)... 2606:50c0:8000::153, 2606:50c0:8001::153, 2606:50c0:8002::153, ...
Connecting to nlp100.github.io (nlp100.github.io)|2606:50c0:8000::153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 975789 (953K) [text/plain]
Saving to: 'data/neko.txt'

neko.txt            100%[===================>] 952.92K  --.-KB/s    in 0.1s    

2023-04-18 22:13:50 (7.85 MB/s) - 'data/neko.txt' saved [975789/975789]



In [8]:
# 先頭10行表示
!cat -n data/neko.txt | head

     1	一
     2	
     3	　吾輩は猫である。
     4	名前はまだ無い。
     5	
     6	　どこで生れたかとんと見当がつかぬ。
     7	何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。
     8	吾輩はここで始めて人間というものを見た。
     9	しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。
    10	この書生というのは時々我々を捕えて煮て食うという話である。
cat: stdout: Broken pipe


In [35]:
# MeCabを使ってみる
import MeCab, ipadic

# そのまま使うと単語ごとの情報になる
mecab = MeCab.Tagger(ipadic.MECAB_ARGS)
text = '今日はいい天気ですね。'
print(mecab.parse(text))

# 分かち書きにする（語の区切りにSpaveを入れる）
import MeCab
wakati = MeCab.Tagger("-Owakati")
print(wakati.parse(text))
print(wakati.parse(text).split())  # リストにする


今日	名詞,副詞可能,*,*,*,*,今日,キョウ,キョー
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
いい	形容詞,自立,*,*,形容詞・イイ,基本形,いい,イイ,イイ
天気	名詞,一般,*,*,*,*,天気,テンキ,テンキ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
ね	助詞,終助詞,*,*,*,*,ね,ネ,ネ
。	記号,句点,*,*,*,*,。,。,。
EOS

今日 は いい 天気 です ね 。 

['今日', 'は', 'いい', '天気', 'です', 'ね', '。']


In [36]:
# 形態素解析データをファイルに保存
import MeCab, ipadic

mecab = MeCab.Tagger(ipadic.MECAB_ARGS)
with open('work/neko.txt.mecab', 'w') as fw:
    with open('data/neko.txt') as fr:
        for line in fr:
            line = line.rstrip()
            m = mecab.parse(line)
            fw.write(m)
# 先頭だけ表示
!cat work/neko.txt.mecab | head

一	名詞,数,*,*,*,*,一,イチ,イチ
EOS
EOS
　	記号,空白,*,*,*,*,　,　,　
吾輩	名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
猫	名詞,一般,*,*,*,*,猫,ネコ,ネコ
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ
ある	助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル
。	記号,句点,*,*,*,*,。,。,。
cat: stdout: Broken pipe


mecab.parseの形式は  

表層語\t品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音  

表装語は1番目, 品詞は2番目, 品詞細分類は3番目, 原型は8番目

In [47]:
# 正規表現を用いて取り出す
import re
pattern = r'(.*?)\t(.*?)\,(.*?)\,(.*?)\,(.*?)\,(.*?)\,(.*?)\,(.*?)\,(.*?)\,(.*?)'
repattern = re.compile(pattern)

morpheme_list = [] # 答えとなる形態素リスト
with open('work/neko.txt.mecab') as f:
    for line in f:
        line = line.rstrip() # 改行削除
        match = repattern.match(line)
        if match: # ここでEOSの行は排除する
            surfase = match.group(1) # 表層語
            base = match.group(8)    # 原型
            pos = match.group(2)     # 品詞
            pos1 = match.group(3)    # 品詞細分類1
            morpheme_dict = {'surfase' : surfase, 'base' : base, 'pos' : pos, 'pos1' : pos1} # 辞書に格納する
            morpheme_list.append(morpheme_dict) # リストにいれる

# 10個だけ表示
from itertools import islice
for morpheme in islice(morpheme_list, 10):
    print(morpheme)

{'surfase': '一', 'base': '一', 'pos': '名詞', 'pos1': '数'}
{'surfase': '\u3000', 'base': '\u3000', 'pos': '記号', 'pos1': '空白'}
{'surfase': '吾輩', 'base': '吾輩', 'pos': '名詞', 'pos1': '代名詞'}
{'surfase': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'}
{'surfase': '猫', 'base': '猫', 'pos': '名詞', 'pos1': '一般'}
{'surfase': 'で', 'base': 'だ', 'pos': '助動詞', 'pos1': '*'}
{'surfase': 'ある', 'base': 'ある', 'pos': '助動詞', 'pos1': '*'}
{'surfase': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}
{'surfase': '名前', 'base': '名前', 'pos': '名詞', 'pos1': '一般'}
{'surfase': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'}


<h2 id="31-動詞">31. 動詞</h2>
<p>動詞の表層形をすべて抽出せよ．</p>


In [54]:
verb_list = []    # 動詞を格納するリスト
verb_set = set()  # 動詞を格納する集合
for morpheme in morpheme_list:
    if morpheme['pos'] == '動詞':
        verb_list.append(morpheme['surfase'])
        verb_set.add(morpheme['surfase'])

# 表示
from itertools import islice
for v in islice(verb_list, 10):
    print(v) 

print(f'動詞の数　：{len(verb_list)}')
print(f'動詞の種類：{len(verb_set)}')

生れ
つか
し
泣い
し
いる
始め
見
聞く
捕え
動詞の数　：28905
動詞の種類：3893


<h2 id="32-動詞の基本形">32. 動詞の基本形</h2>
<p>動詞の基本形をすべて抽出せよ．</p>


In [55]:
verb_base_list = []   # リスト
verb_base_set = set() # 集合
for morpheme in morpheme_list:
    if morpheme['pos'] == '動詞':
        verb_base_list.append(morpheme['base'])
        verb_base_set.add(morpheme['base'])

# 表示
from itertools import islice
for v in islice(verb_base_list, 10):
    print(v) 
            
print(f'動詞の基本形の数　：{len(verb_base_list)}')
print(f'動詞の基本形の種類：{len(verb_base_set)}')

生れる
つく
する
泣く
する
いる
始める
見る
聞く
捕える
動詞の基本形の数　：28905
動詞の基本形の種類：2300


<h2 id="33-aのb">33. 「AのB」</h2>
<p>2つの名詞が「の」で連結されている名詞句を抽出せよ．</p>


<h2 id="34-名詞の連接">34. 名詞の連接</h2>
<p>名詞の連接（連続して出現する名詞）を最長一致で抽出せよ．</p>


<h2 id="35-単語の出現頻度">35. 単語の出現頻度</h2>
<p>文章中に出現する単語とその出現頻度を求め，出現頻度の高い順に並べよ．</p>


<h2 id="36-頻度上位10語">36. 頻度上位10語</h2>
<p>出現頻度が高い10語とその出現頻度をグラフ（例えば棒グラフなど）で表示せよ．</p>


<h2 id="37-猫と共起頻度の高い上位10語">37. 「猫」と共起頻度の高い上位10語</h2>
<p>「猫」とよく共起する（共起頻度が高い）10語とその出現頻度をグラフ（例えば棒グラフなど）で表示せよ．</p>


<h2 id="38-ヒストグラム">38. ヒストグラム</h2>
<p>単語の出現頻度のヒストグラムを描け．ただし，横軸は出現頻度を表し，1から単語の出現頻度の最大値までの線形目盛とする．縦軸はx軸で示される出現頻度となった単語の異なり数（種類数）である．</p>


<h2 id="39-zipfの法則">39. Zipfの法則</h2>
<p>単語の出現頻度順位を横軸，その出現頻度を縦軸として，両対数グラフをプロットせよ．</p>
